In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Load the dataset
file_path = '/Users/aliahmed/Downloads/archive/spotify_songs.csv'
data = pd.read_csv(file_path)

# Display the first few rows and the columns of the dataset
print(data.info())
print(data.head())

# Preprocess the text data (track_name and track_artist)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = re.sub(r'@\w+', '', text)  # Remove mentions
        text = re.sub(r'#\w+', '', text)  # Remove hashtags
        text = re.sub(r'\d+', '', text)  # Remove digits
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
        text = ' '.join([word for word in text.split() if word not in stop_words])
    else:
        text = ""
    return text

data['combined_text'] = data['track_name'] + ' ' + data['track_artist']
data['cleaned_text'] = data['combined_text'].apply(preprocess_text)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aliahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import gensim
import gensim.corpora as corpora
from pprint import pprint

# Tokenize the text
def tokenize(text):
    return text.split()

data['tokens'] = data['cleaned_text'].apply(tokenize)

# Create Dictionary
id2word = corpora.Dictionary(data['tokens'])

# Create Corpus
texts = data['tokens']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [7]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

# Print the topics
pprint(lda_model.print_topics())


[(0,
  '0.052*"dont" + 0.036*"chris" + 0.028*"time" + 0.025*"good" + 0.024*"let" + '
  '0.023*"party" + 0.018*"stop" + 0.015*"high" + 0.015*"big" + 0.015*"new"'),
 (1,
  '0.164*"original" + 0.094*"like" + 0.052*"dimitri" + 0.050*"vegas" + '
  '0.024*"chainsmokers" + 0.019*"alesso" + 0.015*"ww" + 0.014*"walker" + '
  '0.012*"better" + 0.011*"axwell"'),
 (2,
  '0.059*"steve" + 0.039*"aoki" + 0.026*"thomas" + 0.020*"julian" + '
  '0.019*"cant" + 0.015*"light" + 0.015*"know" + 0.014*"hold" + 0.014*"music" '
  '+ 0.014*"da"'),
 (3,
  '0.051*"night" + 0.042*"dj" + 0.038*"lights" + 0.038*"get" + 0.035*"zedd" + '
  '0.020*"dirty" + 0.015*"body" + 0.014*"city" + 0.013*"cole" + 0.012*"snake"'),
 (4,
  '0.038*"john" + 0.032*"young" + 0.020*"beautiful" + 0.018*"vivo" + '
  '0.018*"mars" + 0.017*"de" + 0.016*"silva" + 0.015*"hey" + 0.015*"legend" + '
  '0.015*"wont"'),
 (5,
  '0.313*"remix" + 0.024*"one" + 0.023*"rhab" + 0.016*"world" + 0.014*"vs" + '
  '0.011*"klaas" + 0.011*"anthem" + 0.010*"kask

In [8]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.409279  0.117474       1        1  16.188684
9      0.058187 -0.349400       2        1  15.718663
5      0.017302 -0.139501       3        1  14.324383
8     -0.061409  0.057845       4        1   9.203817
0     -0.065859  0.055203       5        1   8.599394
2     -0.070858  0.053518       6        1   8.217333
1     -0.056830  0.066978       7        1   7.905077
3     -0.076624  0.046217       8        1   7.008507
7     -0.072500  0.047429       9        1   6.546279
4     -0.080687  0.044237      10        1   6.287861, topic_info=           Term         Freq        Total Category  logprob  loglift
7         remix  5894.000000  5894.000000  Default  30.0000  30.0000
29         feat  4306.000000  4306.000000  Default  29.0000  29.0000
46         edit  2687.000000  2687.000000  Default  28.0000  28.0000
324    original  1710.000000  1710.000000  Default  27.0000  27.0000
48        radio  2302.000000  2302.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2096       moon    69.504911    70.413507  Topic10  -4.7799   2.7536
2027        war    66.103849    67.012462  Topic10  -4.8300   2.7529
3            ed    64.239061    65.147613  Topic10  -4.8586   2.7525
8       sheeran    62.691524    63.600082  Topic10  -4.8830   2.7522
20827  dahlbäck    78.139261    79.321964  Topic10  -4.6628   2.7515

[353 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1839      9  0.987492      adam
754       2  0.997409  afrojack
728       7  0.992431      alan
3463      8  0.986066     alarm
348       7  0.995015    alesso
...     ...       ...       ...
9283      5  0.104998        ww
9283      7  0.886648        ww
236       2  0.995990         x
667      10  0.995616     young
339       8  0.998047      zedd

[352 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 6, 9, 1, 3, 2, 4, 8, 5])

In [9]:
# Save the visualization as an HTML file
pyLDAvis.save_html(vis, 'lda_vis.html')
print("LDA visualization saved as lda_vis.html")


LDA visualization saved as lda_vis.html
